In [ ]:
import jigsolver
import numpy as np
import matplotlib.pyplot as plt
from jigsolver.puzzle import *
from copy import copy, deepcopy

In [ ]:
np.random.seed(0)
P=Piece(np.random.random(size=(3,3,3)))
Q=Piece(np.random.random(size=(3,3,3)))
plt.imshow(P.picture)
plt.title('Example of a Piece')
plt.show()

plt.imshow(P.bottom_side)
plt.title('Bottom border of the piece')

In [ ]:
img_folder = 'img/'
img = plt.imread(img_folder + "eiffel.jpg")

In [ ]:
eiffel_puzzle = Puzzle(patch_size=100)
eiffel_puzzle.create_from_img(img)
eiffel_puzzle.display()

In [ ]:
eiffel_unsolved = copy(eiffel_puzzle)
eiffel_unsolved.shuffle()

In [ ]:
some_piece = eiffel_unsolved.bag_of_pieces[0]
eiffel_unsolved.board[3,2] = some_piece
some_piece = eiffel_unsolved.bag_of_pieces[1]
eiffel_unsolved.board[2,2] = some_piece

In [ ]:
eiffel_unsolved.display()

In [ ]:
for slot in eiffel_unsolved.board.neighbors(3,2):
    print(slot)